# 🏛️ Semantic Objects - Basic Tutorial

Welcome to the **Semantic Objects** library! This tutorial will walk you through the core functionality step by step.

## What You'll Learn

1. **Creating semantic objects** - Define spaces, windows, and properties
2. **Working with properties** - Handle values, units, and quantity kinds
3. **Template generation** - Export BuildingMOTIF templates
4. **SHACL validation** - Generate validation rules
5. **Model building** - Create RDF models with BMotifSession

Let's get started! 🚀

## Setup and Imports

In [1]:
# Core imports
from semantic_objects.s223 import *
from semantic_objects.discovery import get_related_classes
from semantic_objects.exporters import export_templates
from semantic_objects.build_model import BMotifSession
from pprint import pprint
import pandas as pd

print("✅ Imports successful!")

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


✅ Imports successful!


## 1. 🔍 Exploring Entities and Properties

In this library, **Entities** (like a Space or Window) represent physical or logical objects, while **Properties** define the attributes associated with them.

### Inspecting an Entity

Let's look at what fields are available for a `Space`:

In [2]:
# View the fields associated with a Space
print("Space fields:")
pprint(Space.__dataclass_fields__)

print("\n" + "="*50)
print("Window fields:")
pprint(Window.__dataclass_fields__)

Space fields:
{'area': Field(name='area',type=<class 'semantic_objects.s223.properties.Area'>,default=<dataclasses._MISSING_TYPE object at 0x10542f650>,default_factory=<dataclasses._MISSING_TYPE object at 0x10542f650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None, 'value': None}),kw_only=False,_field_type=_FIELD)}

Window fields:
{'area': Field(name='area',type=<class 'semantic_objects.s223.properties.Area'>,default=<dataclasses._MISSING_TYPE object at 0x10542f650>,default_factory=<dataclasses._MISSING_TYPE object at 0x10542f650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None, 'value': None}),kw_only=False,_field_type=_FIELD),
 'azimuth': Field(name='azimuth',type=<class 'semantic_objects.s223.properties.Azimuth'>,default=<dataclasses._MISSING_TYPE object at 0x10542f65

### Diving into Property Metadata

Properties are defined with specific values, units, and **Quantity Kinds**:

In [3]:
# Inspect the metadata of the Area property
print(f"Property Class: {Area}")
print("\nArea property fields:")
pprint(Area.__dataclass_fields__)

# Look at the quantity kind
print(f"\nArea quantity kind: {Area.qk}")
print(f"Quantity kind IRI: {Area.qk._get_iri()}")

Property Class: <class 'semantic_objects.s223.properties.Area'>

Area property fields:
{'qk': Field(name='qk',type=<class 'semantic_objects.qudt.quantitykinds.QuantityKind'>,default=<class 'semantic_objects.qudt.quantitykinds.Area'>,default_factory=<dataclasses._MISSING_TYPE object at 0x10542f650>,init=False,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': False, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'unit': Field(name='unit',type=typing.Optional[semantic_objects.units.Unit],default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x10542f650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True}),kw_only=False,_field_type=_FIELD),
 'value': Field(name='value',type=<class 'float'>,default=<dataclasses._MISSING_TYPE object at 0x10542f650>,default_factory=<dataclasses._MISSING_TYPE object at 0x10542f650>,init=True,rep

## 2. 🏗️ Creating Semantic Objects

Let's create some semantic objects and see how they work:

In [4]:
# Create a simple space with area
office = Space(area=150.0)  # Area in default units (ft²)

print(f"Created space with area: {office.area.value} {office.area.unit}")
print(f"Space name: {office._name}")
print(f"Area quantity kind: {office.area.qk}")

Created space with area: 150.0 <class 'semantic_objects.qudt.units.M2'>
Space name: Space_1
Area quantity kind: <class 'semantic_objects.qudt.quantitykinds.Area'>


In [5]:
# Create a window with multiple properties
south_window = Window(
    area=25.0,      # ft²
    azimuth=180.0,  # degrees (south-facing)
    tilt=90.0       # degrees (vertical)
)

print(f"Window area: {south_window.area.value} {south_window.area.unit}")
print(f"Window azimuth: {south_window.azimuth.value}° (south-facing)")
print(f"Window tilt: {south_window.tilt.value}° (vertical)")
print(f"Window name: {south_window._name}")

Window area: 25.0 <class 'semantic_objects.qudt.units.M2'>
Window azimuth: 180.0° (south-facing)
Window tilt: 90.0° (vertical)
Window name: Window_1


### Working with Different Units

In [6]:
# Import specific units
from semantic_objects.qudt.units import M2, FT2

# Create area with explicit metric units
metric_area = Area(50.0, unit=M2)
imperial_area = Area(50.0, unit=FT2)

print(f"Metric area: {metric_area.value} {metric_area.unit}")
print(f"Imperial area: {imperial_area.value} {imperial_area.unit}")

# Create space with metric area
metric_space = Space(area=metric_area)
print(f"\nMetric space area: {metric_space.area.value} {metric_space.area.unit}")

Metric area: 50.0 <class 'semantic_objects.qudt.units.M2'>
Imperial area: 50.0 <class 'semantic_objects.qudt.units.FT2'>

Metric space area: 50.0 <class 'semantic_objects.qudt.units.M2'>


## 3. 📝 Template Generation

A core feature is exporting these objects into **YAML templates** for use by `buildingmotif`:

In [7]:
# Export templates to a directory
export_templates(Space, 'templates')

print("✅ Templates exported to 'templates/' directory")
print("Generated files:")
import os
for file in os.listdir('templates'):
    print(f"  - {file}")

✅ Templates exported to 'templates/' directory
Generated files:
  - entities.yml
  - values.yml


### Inspecting Generated Templates

Let's see what the generated templates look like:

In [8]:
# Get related classes for Space
related_classes = get_related_classes(Space)
predicates, entities = related_classes

print(f"Found {len(predicates)} predicates, {len(entities)} entities")
print("\nEntities:")
for entity in entities:
    print(f"  - {entity.__name__}")

Found 0 predicates, 3 entities

Entities:
  - Area
  - Space
  - QuantityKind


In [9]:
# Look at a generated YAML template
print("Example generated YAML template for Space:")
print(Space.generate_yaml_template())

Example generated YAML template for Space:
{'Space': {'body': '@prefix P: <urn:___param___#> .\n@prefix s223: <http://data.ashrae.org/standard223#> .\n\nP:name a s223:Space .\n\n', 'dependencies': [{'template': 'Area', 'args': {'name': 'area'}}]}}


## 4. ⚖️ Validation and SHACL Generation

The library can generate **RDF class definitions** and **SHACL shapes** to ensure your models adhere to ontology constraints:

In [10]:
# Generate RDF definition including the parent hierarchy (RDFS inheritance)
print("=== Full Hierarchy ===")
full_hierarchy = Space.generate_rdf_class_definition(include_hierarchy=True)
print(full_hierarchy[:500] + "..." if len(full_hierarchy) > 500 else full_hierarchy)

=== Full Hierarchy ===
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

s223:Space a s223:Class,
        rdfs:Class,
        sh:NodeShape ;
    rdfs:label "Domain Space" ;
    rdfs:subClassOf s223:DomainSpace ;
    sh:property [ a sh:PropertyShape ;
            rdfs:comment "If the relation `contains` is present it must associate the `Space` with a `PhysicalSpace`...


In [11]:
# Generate just the local class constraints
print("=== Local Constraints Only ===")
local_constraints = Space.generate_rdf_class_definition(include_hierarchy=False)
print(local_constraints)

=== Local Constraints Only ===
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

s223:Space a s223:Class,
        rdfs:Class,
        sh:NodeShape ;
    rdfs:label "Domain Space" ;
    rdfs:subClassOf s223:DomainSpace ;
    sh:property [ a sh:PropertyShape ;
            rdfs:comment "If the relation `hasProperty` is present it must associate the `Space` with a `Area`." ;
            sh:minCount 1 ;
            sh:path s223:hasProperty ;
            sh:qualifiedMinCount 1 ;
            sh:qualifiedValueShape [ a sh:NodeShape ;
                    rdfs:label "None" ;
                    sh:class s223:Area ] ;
            sh:value s223:Area ] .




### SPARQL Query Generation

The library automatically generates SPARQL queries from class definitions:

In [12]:
# Generate SPARQL query for Space class
space_query = Space.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Space:")
print(space_query)

Generated SPARQL query for Space:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX s223: <http://data.ashrae.org/standard223#>
SELECT DISTINCT * WHERE { ?area rdf:type s223:QuantifiableObservableProperty .
?name rdf:type s223:Space .
?name s223:hasProperty ?area .
?area qudt:hasQuantityKind quantitykind:Area .
FILTER NOT EXISTS { ?area <http://data.ashrae.org/standard223#hasAspect> ?area_aspects_in } }


In [13]:
# Generate query for Window (more complex with multiple properties)
window_query = Window.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Window:")
print(window_query)

Generated SPARQL query for Window:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX s223: <http://data.ashrae.org/standard223#>
SELECT DISTINCT * WHERE { ?name rdf:type s223:Window .
?tilt qudt:hasQuantityKind quantitykind:Tilt .
?name s223:hasProperty ?azimuth .
?area rdf:type s223:QuantifiableObservableProperty .
?azimuth rdf:type s223:QuantifiableObservableProperty .
?azimuth qudt:hasQuantityKind quantitykind:Azimuth .
?name s223:hasProperty ?tilt .
?tilt rdf:type s223:QuantifiableObservableProperty .
?name s223:hasProperty ?area .
?area qudt:hasQuantityKind quantitykind:Area .
FILTER NOT EXISTS { ?area <http://data.ashrae.org/standard223#hasAspect> ?area_aspects_in }
FILTER NOT EXISTS { ?azimuth <http://data.ashrae.org/standard223#hasAspect> ?azimuth_aspects_in }
FILTER NOT EXISTS { ?tilt <http://data.ashrae.org/standard223#hasAspect> ?tilt_aspects_in } }


## 5. 🏗️ Building Models with BMotifSession

The `BMotifSession` class integrates with BuildingMOTIF to create RDF models:

In [14]:
# Create a BMotifSession
session = BMotifSession(ns='tutorial')

# Load class templates
session.load_class_templates(Space)

print("✅ BMotifSession created and templates loaded")
print(f"Available templates: {list(session.templates.keys())}")

DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:tutorial#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:tutorial#', with graph: 'eb6a59da-394f-4c60-bb5a-7f94139cb57c'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'eb6a59da-394f-4c60-bb5a-7f94139cb57c' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Area'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'f8a9e88d-9671-4a8e-b060-c6143768febc' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '9e0558ae-0c9a

✅ BMotifSession created and templates loaded
Available templates: ['Area', 'Space', 'QuantityKind']


In [15]:
# Look at template parameters
space_template = session.templates['Space']
print("Space template parameters:")
print(space_template.inline_dependencies().all_parameters)

Space template parameters:
{'name', 'area'}


In [16]:
# Create a space and evaluate it
conference_room = Space(area=200.0)
conference_room._name = "ConferenceRoom_101"

print("Space field values:")
pprint(conference_room.get_field_values(recursive=True))

Space field values:
{'_name': 'ConferenceRoom_101',
 'area': {'_name': 'Area_5',
          '_type': 'Area',
          'qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'),
          'unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2'),
          'value': 200.0}}


In [17]:
# Evaluate the space (generate RDF)
session.evaluate(conference_room)

print("✅ Space evaluated and added to model")
print("\nGenerated RDF (first 1000 characters):")
rdf_output = session.graph.serialize(format='turtle')
print(rdf_output[:1000] + "..." if len(rdf_output) > 1000 else rdf_output)

{'name': rdflib.term.URIRef('urn:tutorial#ConferenceRoom_101'), 'area': rdflib.term.URIRef('urn:tutorial#Area_5'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'), 'area-value': rdflib.term.Literal('200.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), 'area-unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2')}
✅ Space evaluated and added to model

Generated RDF (first 1000 characters):
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:tutorial#> a owl:Ontology .

<urn:tutorial#Area_5> a <http://data.ashrae.org/standard223#Area> .

<urn:tutorial#ConferenceRoom_101> a <http://data.ashrae.org/standard223#Space> .




## 6. 🔄 Working with Multiple Objects

Let's create a more complex example with multiple spaces and windows:

In [18]:
# Create multiple spaces
spaces = [
    Space(area=150.0),  # Office 1
    Space(area=200.0),  # Conference room
    Space(area=100.0),  # Office 2
]

# Set names
space_names = ["Office_101", "ConferenceRoom_201", "Office_102"]
for space, name in zip(spaces, space_names):
    space._name = name

print("Created spaces:")
for space in spaces:
    print(f"  - {space._name}: {space.area.value} {space.area.unit}")

Created spaces:
  - Office_101: 150.0 <class 'semantic_objects.qudt.units.M2'>
  - ConferenceRoom_201: 200.0 <class 'semantic_objects.qudt.units.M2'>
  - Office_102: 100.0 <class 'semantic_objects.qudt.units.M2'>


In [19]:
# Create windows for the spaces
windows = [
    Window(area=20.0, azimuth=90.0, tilt=90.0),   # East-facing
    Window(area=30.0, azimuth=180.0, tilt=90.0),  # South-facing
    Window(area=15.0, azimuth=270.0, tilt=90.0),  # West-facing
]

window_names = ["Window_E101", "Window_S201", "Window_W102"]
for window, name in zip(windows, window_names):
    window._name = name

print("Created windows:")
for window in windows:
    direction = {90.0: "East", 180.0: "South", 270.0: "West"}[window.azimuth.value]
    print(f"  - {window._name}: {window.area.value} ft², {direction}-facing")

Created windows:
  - Window_E101: 20.0 ft², East-facing
  - Window_S201: 30.0 ft², South-facing
  - Window_W102: 15.0 ft², West-facing


In [20]:
# Create a new session and evaluate all objects
building_session = BMotifSession(ns='building')

# Load templates for both Space and Window
building_session.load_class_templates(Space)
building_session.load_class_templates(Window)

print(f"Available templates: {list(building_session.templates.keys())}")

DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:building#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:building#', with graph: 'd466cfd5-434f-48e2-ac8d-abd95d3af4f0'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'd466cfd5-434f-48e2-ac8d-abd95d3af4f0' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Area'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '4dcfb94f-d2e2-4f35-a74b-b442ac648cf9' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '2b4e145c-3c1e-44d3-8d43-13159ef6d000' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'QuantityKind'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'a041156c-b191-4764-a210-93606d60986f' in database with: 0 triples
DEBU

Available templates: ['Area', 'Space', 'QuantityKind', 'Tilt', 'Azimuth', 'Window']


In [21]:
# Evaluate all spaces and windows
for space in spaces:
    building_session.evaluate(space)
    
for window in windows:
    building_session.evaluate(window)

print("✅ All objects evaluated")

# Count triples in the graph
triple_count = len(building_session.graph)
print(f"Generated {triple_count} RDF triples")

{'name': rdflib.term.URIRef('urn:building#Office_101'), 'area': rdflib.term.URIRef('urn:building#Area_6'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'), 'area-value': rdflib.term.Literal('150.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), 'area-unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2')}
{'name': rdflib.term.URIRef('urn:building#ConferenceRoom_201'), 'area': rdflib.term.URIRef('urn:building#Area_7'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'), 'area-value': rdflib.term.Literal('200.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), 'area-unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2')}
{'name': rdflib.term.URIRef('urn:building#Office_102'), 'area': rdflib.term.URIRef('urn:building#Area_8'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef(

In [22]:
# Show a sample of the generated RDF
print("Sample of generated RDF:")
rdf_sample = building_session.graph.serialize(format='turtle')
lines = rdf_sample.split('\n')
print('\n'.join(lines[:30]))  # Show first 30 lines
if len(lines) > 30:
    print(f"... and {len(lines) - 30} more lines")

Sample of generated RDF:
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:building#> a owl:Ontology .

<urn:building#Area_10> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Area_11> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Area_6> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Area_7> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Area_8> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Area_9> a <http://data.ashrae.org/standard223#Area> .

<urn:building#Azimuth_2> a <http://data.ashrae.org/standard223#Azimuth> .

<urn:building#Azimuth_3> a <http://data.ashrae.org/standard223#Azimuth> .

<urn:building#Azimuth_4> a <http://data.ashrae.org/standard223#Azimuth> .

<urn:building#ConferenceRoom_201> a <http://data.ashrae.org/standard223#Space> .

<urn:building#Office_101> a <http://data.ashrae.org/standard223#Space> .

<urn:building#Office_102> a <http://data.ashrae.org/standard223#Space> .

<urn:building#Tilt_2>

## 7. 📊 Summary and Next Steps

Congratulations! You've learned the basics of Semantic Objects:

✅ **Created semantic objects** (Space, Window) with properties (Area, Azimuth, Tilt)  
✅ **Worked with units** and quantity kinds  
✅ **Generated templates** for BuildingMOTIF  
✅ **Created SHACL shapes** for validation  
✅ **Built RDF models** using BMotifSession  
✅ **Generated SPARQL queries** automatically  

### What's Next?

1. **Model Loading Tutorial**: Learn to load RDF data into Python objects
2. **Advanced Examples**: Complex relationships and custom entities
3. **Template Generation**: Deep dive into BuildingMOTIF integration
4. **Custom Entities**: Create your own semantic object types

### Key Takeaways

- **Pythonic Interface**: Define ontology concepts as simple dataclasses
- **Automatic Generation**: SPARQL queries, SHACL shapes, and templates generated automatically
- **Type Safety**: Full type hints and validation
- **Flexible Units**: Support for metric and imperial units
- **BuildingMOTIF Integration**: Seamless template generation and model building

Happy modeling! 🎉